<a href="https://colab.research.google.com/github/Dpgofast/DS-Sprint-01-Dealing-With-Data/blob/master/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [3]:
# TODO - your code here
# Use what we did live in lecture as an example
import numpy as np

def neural(m1,m2,w1,w2,b):
  z= m1 * w1 + m2 * w2 +b
  return sigmoid(z)
# A function that lays a baseline for a neural network.
def sigmoid(x):
  return 1/(1 + np.exp(-x))
# defines sigmoid function called above 

w1= np.random.randn()
w2= np.random.randn()
b= np.random.randn()
# filling variables with random numbers for example
    
neural(2,3,w1,w2,b)

0.07064259576444483

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.
    The code above shows the first rudimentary steps to a basic neural network. This particular model although not visually stimulating does draw inferences from given variables in order to predict a category of best fit. For example the object could be classified by color given some variables like: hieght, weight, depth, age. This relatively simple block of code when elaborated on could possibly make predictions not that otherwise have remained hidden. 

2.  What was the most challenging part of what you did?
    The most callenging part for me was finding the idea. I had many thoughts but the execution and explaining in a non-technical way seemed most efficient with this example. 

3.  What was the most interesting thing you learned?
    The most interesting thing I learned would be the underlying simplicity to begin a neural network. Up to this point I believed it would require some new code that would be unreadable for me at this stage, but the math is the only thing that really requires deep thought. 

4.  What area would you like to explore with more time? I would enjoy Learning more Machine Learning, and how to effectively deploy neural networks to predict, solve and draw new conclusions from data. 




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about